In [1]:
import os
import sys
currentdir = os.path.dirname(os.path.realpath('__file__'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

In [2]:
from norm import *
import requests

In [2]:
content = requests.get('https://raw.githubusercontent.com/108michael/congress-legislators/master/legislators-historical.yaml').content
source = yaml.load(content)

In [185]:
df = pd.DataFrame(source)
df['index'] = df.index
df_family = df[~df.family.isna()]
df_family.head()

,bio,family,id,leadership_roles,name,other_names,terms,index
11737,"{'birthday': '1962-03-30', 'gender': 'M', 'ind...","[{'name': 'Nicholas J. Begich', 'relation': 's...","{'bioguide': 'B001265', 'thomas': '01898', 'li...",NaN,"{'first': 'Mark', 'last': 'Begich', 'official_...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':...",11737
11739,"{'birthday': '1953-05-26', 'gender': 'F', 'ind...","[{'name': 'Lawton M. Chiles Jr.', 'relation': ...","{'bioguide': 'H001049', 'thomas': '01902', 'li...",NaN,"{'first': 'Kay', 'last': 'Hagan', 'official_fu...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':...",11739
11743,"{'birthday': '1934-06-28', 'gender': 'M', 'rel...","[{'name': 'Sander Martin Levin', 'relation': '...","{'bioguide': 'L000261', 'thomas': '01384', 'li...",NaN,"{'first': 'Carl', 'last': 'Levin', 'official_f...",NaN,"[{'type': 'sen', 'start': '1979-01-15', 'end':...",11743
11744,"{'birthday': '1963-01-10', 'gender': 'M', 'rel...","[{'name': 'David H. Pryor', 'relation': 'son',...","{'bioguide': 'P000590', 'thomas': '01701', 'li...",NaN,"{'first': 'Mark', 'last': 'Pryor', 'official_f...",NaN,"[{'type': 'sen', 'start': '2003-01-07', 'end':...",11744
11745,"{'birthday': '1937-06-18', 'gender': 'M', 'rel...","[{'name': 'Nelson Aldrich Rockefeller', 'relat...","{'bioguide': 'R000361', 'thomas': '01424', 'li...",NaN,"{'first': 'John', 'middle': 'D.', 'last': 'Roc...",NaN,"[{'type': 'sen', 'start': '1985-01-03', 'end':...",11745
